## 1_でクロールしたpicleファイルをロードし、一つにまとめて、解析に使いやすいようにデータを整形したうえで再度pickleファイルで保存
- ファイルの読み込み元：'./crawledData/　以下
- ファイルの保存先：'./arrangedData/　以下

In [1]:
import glob
import pandas as pd

#### レース情報をまとめたpicleファイルを読み込んで一つにまとめる

In [2]:
# ファイル内容確認用
all_files = glob.glob('./data/crawledData/*.pkl')

# その日のレース情報をまとめたdfを要素にもつリスト
race_df_list = [pd.read_pickle(file) for file in all_files]
# concatして一つのdfにまとめる
race_df = pd.concat(race_df_list)
race_df

racer_id racer_class racer_name num_false_start  \
date       venue raceNumber 枠                                                   
2020-11-03 桐　生   1          1     4851          A1     関　　　浩哉              F0   
                            2     4362          A1     土屋　　智則              F0   
                            3     4080          A1     山崎　　哲司              F0   
                            4     3554          A1     仲口　　博崇              F0   
                            5     4504          A1     前田　　将太              F0   
...                                ...         ...        ...             ...   
2020-11-26 福　岡   12         2     4816          A1     村松　　修二              F0   
                            3     4737          A1     竹井　　貴史              F1   
                            4     4643          A2     近江　　翔吾              F0   
                            5     4802          A1     中田　　達也              F0   
                            6     4960          A2     黒野　　元基              F0   

                              num_late_start motorNo モーター2連率 モーター3連率 boatNo  \
date       venue raceNumber 枠                                                 
2020-11-03 桐　生   1          1             L0      73   33.33   46.20     61   
                            2             L0      20   43.98   61.45     14   
                            3             L0      24   38.20   61.24     40   
                            4             L0      23   34.39   47.51     59   
                            5             L0      59   47.73   63.64     70   
...                                      ...     ...     ...     ...    ...   
2020-11-26 福　岡   12         2             L0      70   25.97   48.05    135   
                            3             L0      47   26.25   38.75    116   
                            4             L0      57   24.68   40.26    120   
                            5             L0      67   31.46   50.56    174   
                            6             L0      14   42.03   60.87    106   

                               ボート2連率  ... weight exhibitionTime  tilt  \
date       venue raceNumber 枠          ...                               
2020-11-03 桐　生   1          1   28.49  ...   49.5           6.74  -0.5   
                            2   35.91  ...   51.1           6.67  -0.5   
                            3   35.71  ...   51.0           6.70  -0.5   
                            4   41.01  ...   51.4           6.76  -0.5   
                            5   34.64  ...   52.8           6.82  -0.5   
...                               ...  ...    ...            ...   ...   
2020-11-26 福　岡   12         2   46.97  ...   49.5           6.86  -0.5   
                            3   36.84  ...   51.5           6.78  -0.5   
                            4   30.26  ...   51.5           6.86  -0.5   
                            5   32.89  ...   51.5           6.81   0.0   
                            6   37.35  ...   52.3           6.79  -0.5   

                              exhibition_cource exhibition_ST flying late 着順  \
date       venue raceNumber 枠                                                  
2020-11-03 桐　生   1          1                 1           .19    0.0    0  １   
                            2                 2           .15    0.0    0  ４   
                            3                 3           .09    0.0    0  ３   
                            4                 4           .17    0.0    0  ２   
                            5                 5           .18    0.0    0  ５   
...                                         ...           ...    ...  ... ..   
2020-11-26 福　岡   12         2                 2           .04    0.0    0  １   
                            3                 3           .07    0.0    0  ３   
                            4                 4           .08    0.0    0  ２   
                            5                 5           .10    0.0    0  ５   
                         

#### 解析に使いやすいようにデータを整形する

In [3]:
from datetime import datetime as dt

race_df_arranged = race_df.copy()

# それぞれの列のデータタイプの変更など
race_df_arranged["boatNo"] = race_df_arranged["boatNo"].astype("int")
race_df_arranged["motorNo"] = race_df_arranged["motorNo"].astype("int")
race_df_arranged["num_false_start"] = race_df_arranged["num_false_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["num_late_start"] = race_df_arranged["num_late_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["racer_id"] = race_df_arranged["racer_id"].astype("int")
race_df_arranged["ボート2連率"] = race_df_arranged["ボート2連率"].astype("float64")
race_df_arranged["ボート3連率"] = race_df_arranged["ボート3連率"].astype("float64")
race_df_arranged["racer_class"] = race_df_arranged["racer_class"].map({"A1": 1, "A2": 2, "B1": 3, "B2": 4})
race_df_arranged["モーター2連率"] = race_df_arranged["モーター2連率"].astype("float64")
race_df_arranged["モーター3連率"] = race_df_arranged["モーター3連率"].astype("float64")
race_df_arranged["exhibition_ST"] = race_df_arranged["exhibition_ST"].astype("float64")
race_df_arranged["tilt"] = race_df_arranged["tilt"].astype("float64")
race_df_arranged["weight"] = race_df_arranged["weight"].astype("float64")

# weatherをdummy変数に変換
dummy_df_weather = pd.get_dummies(race_df_arranged["weather"], prefix='weather', drop_first=True)
race_df_arranged = pd.concat([race_df_arranged, dummy_df_weather], axis=1)

# 何らかの事情でデータから抜いたほうがいいレースを集める
race_df_del_list = []
# 転覆等があり、着順が6までつけられなかったレースを削除. 転覆になった選手の行だけではなくそのレースごと列を削除する
race_df_del_1 = race_df_arranged[race_df_arranged["着順"].astype(str).str.contains("\D")]
race_df_del_1.reset_index(inplace=True)
race_df_del_list.append(race_df_del_1)

# それぞれvalueに''を含むレースを削除
may_have_blank_list = [
    "exhibitionTime", "temperature", "water_temperature",
    "wave_height", "wind_speed", "exhibition_cource"
]
for column in may_have_blank_list:
    race_df_del_ = race_df_arranged[~race_df_arranged[column].str.match("\d.*")]
    race_df_del_.reset_index(inplace=True)
    race_df_del_list.append(race_df_del_)

# Nanを含む列があるレースを削除
race_df_del_nan = race_df_arranged[race_df_arranged.isnull().any(axis=1)]
race_df_del_nan.reset_index(inplace=True)
race_df_del_list.append(race_df_del_nan)

# 消したいレースの開催地・場所・レース番号のリストを作成
remove_race_list_nested = [race_df_del[["date", "venue", "raceNumber"]].values.tolist() for race_df_del in race_df_del_list]
remove_race_list =  [item for l in remove_race_list_nested for item in l]

remove_index_list = list(set([tuple(race) for race in remove_race_list]))
race_df_arranged.drop(index=remove_index_list, inplace=True)

# 着順のカラムをinterger型に変換
race_df_arranged["着順"] = race_df_arranged["着順"].astype("int")

race_df_arranged["exhibitionTime"] = race_df_arranged["exhibitionTime"].astype("float64")
race_df_arranged["temperature"] = race_df_arranged["temperature"].astype("float64")
race_df_arranged["water_temperature"] = race_df_arranged["water_temperature"].astype("float64")
race_df_arranged["wave_height"] = race_df_arranged["wave_height"].astype("int")
race_df_arranged["wind_speed"] = race_df_arranged["wind_speed"].astype("int")
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("int")

# multiindexは扱いがめんどくさいので全部カラムにする
race_df_arranged.reset_index(inplace=True)

# date列をdatetim型に変換
race_df_arranged["date"] = race_df_arranged["date"].map(lambda x: dt.strptime(x, '%Y-%m-%d'))

# dateとraceNumberでsort
race_df_arranged["raceNumber"] = race_df_arranged["raceNumber"].astype("float")
race_df_arranged.sort_values(["date", "venue", "raceNumber", "枠"], inplace=True)

# 統計量の算出を行うためのdf: pickleファイルで保存
race_df_arranged.to_pickle('./data/arrangedData/data_formatted_for_statistic.pkl')

# 予測の際にそのままinputにすることができない列を削除
race_df_arranged.drop(["boatNo", "motorNo", "racer_id", "racer_name", "weather", "タイム", "start_time"], axis=1, inplace=True)

# 予測用のdf: pickleファイルで保存
race_df_arranged.to_pickle('./data/arrangedData/data_formatted_1.pkl')

/Users/grice/opt/anaconda3/envs/boatrace2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [4]:
race_df_arranged

,date,venue,raceNumber,枠,racer_class,num_false_start,num_late_start,モーター2連率,モーター3連率,ボート2連率,...,exhibitionTime,tilt,exhibition_cource,exhibition_ST,flying,late,着順,weather_曇り,weather_雨,weather_雪
11106,2020-11-01,下 関,1.0,1,2,0,0,32.35,52.21,31.25,...,6.67,-0.5,1,0.05,0.0,0,1,1,0,0
11107,2020-11-01,下 関,1.0,2,3,0,0,38.17,60.31,30.53,...,6.73,-0.5,2,0.11,0.0,0,2,1,0,0
11108,2020-11-01,下 関,1.0,3,3,0,0,31.75,47.62,36.22,...,6.81,-0.5,3,0.18,0.0,0,5,1,0,0
11109,2020-11-01,下 関,1.0,4,3,0,0,37.67,49.32,38.28,...,6.79,-0.5,4,0.01,0.0,0,4,1,0,0
11110,2020-11-01,下 関,1.0,5,3,0,0,34.96,49.59,22.58,...,6.81,-0.5,5,0.21,0.0,0,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88471,2021-03-29,蒲 郡,11.0,2,3,0,0,38.92,51.72,32.99,...,6.89,-0.5,2,0.11,0.0,0,4,0,0,0
88472,2021-03-29,蒲 郡,11.0,3,1,0,0,37.32,51.20,40.98,...,6.75,-0.5,3,0.10,0.0,0,1,0,0,0
88473,2021-03-29,蒲 郡,11.0,4,1,0,0,35.55,51.66,44.95,...,6.81,-0.5,4,0.15,0.0,0,6,0,0,0
88474,2021-03-29,蒲 郡,11.0,5,2,1,0,23.91,34.78,27.01,...,6.84,-0.5,5,0.21,0.0,0,3,0,0,0
